- 단계구분도: 경기도내 인구소멸비율

In [1]:
import json, folium
import numpy as np
import pandas as pd

- 경기도 시군구별 인구소멸비율 구하기

In [2]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(3)
pop = pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102
56,시군,경기도 과천시,77775,0.112


- geo data 읽기

In [3]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

- 인구 데이터와 geo 데이터간 인덱시 일치 시키기

In [4]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율,시군구
1,시군,경기도 가평군,62197,0.057,가평군
9,구,경기도 고양시 덕양구,487874,0.107,고양시덕양구
31,구,경기도 고양시 일산동구,296590,0.110,고양시일산동구
44,구,경기도 고양시 일산서구,290738,0.102,고양시일산서구
56,시군,경기도 과천시,77775,0.112,과천시


In [5]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.057
고양시덕양구,구,경기도 고양시 덕양구,487874,0.107
고양시일산동구,구,경기도 고양시 일산동구,296590,0.110
고양시일산서구,구,경기도 고양시 일산서구,290738,0.102
과천시,시군,경기도 과천시,77775,0.112


- 단계 구분도

In [8]:
map1 = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,
    columns=[pop.index, pop.인구소멸비율],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map1)
map1

In [12]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [13]:
gu_dict = get_text_location(geo_data)
for gu_name in pop.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map1)
map1